# Agregação de Qualidade - Isca-aulassemanais

Objetivo: Esse pipeline tem como objetivo isolar os dados de um determinado período da campanha Isca-aulassemanais e fazer as agregações atinentes ao dados de tráfego.

### Agregações propostas:

Isolar dados cuja "utm_source like %ads%" e agregar as utms campaign, medium e content a quantiadde de leads únicos, leads únicos com renda qualificada (lurq) e leads únicos qualificados multi-fatorial (luqmf).

Após a agregação, realizar um merge com as identificações de tráfego de modo a obter as nomenclaturas das campanhas, públicos e anúncios levando em consideração as seguintes equivalências de colunas:


#### 1. utm_campaign =  campaign_id
#### 2. utm_medium =  adset_id
#### 3. utm_content =  ad_id

# Step 0 - Importação de Dependências

In [1]:
# Importações básicas
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
from datetime import datetime
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

#

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df,
    remove_buyers_from_dataframe
)

CRONOGRAMA_SUBDOMAIN = 'cronogramadosfluentes-xwamel'

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


# Step 1: Importação de Origens de Dados.

In [2]:
CAMPAIGN_ID = 'lcto-ofan-jan26'
CONVERSION_TYPE_ID = 1


# Importando dados de leads brutos da Campanha
df = db.execute_query_from_file("select_complete_lead_data_from_campaign", params={"campaign_id": CAMPAIGN_ID, "conversion_type_id": CONVERSION_TYPE_ID})

# Importando lookup Table de Campanhas, Públicos e Anúncios
trf_data = load_raw_data('ofan_trf_data.csv')

# Step 2: Filtrado dados da tabela principal por período e source

In [3]:
# Log do shape antes da filtragem
print(f"Shape do DataFrame antes da filtragem: {df.shape}")

df = df[df['utm_source'].str.contains("ads", regex=False, na=False, case=False)]

# Log do shape após a filtragem
print(f"Shape do DataFrame após a filtragem: {df.shape}")

Shape do DataFrame antes da filtragem: (12484, 19)
Shape do DataFrame após a filtragem: (7178, 19)


# Step 3: Gerando Agregações com contagens

In [4]:

# Agregação das utms: utm_campaign, utm_medium, utm_content
agg_df = (
    df
    .groupby(['last_conversion_date', 'utm_campaign', 'utm_medium', 'utm_content'])
    .agg(
        leads_unicos=('lead_id', 'nunique'),
        lurq=('renda_qualificada', lambda x: df.loc[x.index, 'lead_id'][x].nunique()),
        luqmf=('qualificado_geral', lambda x: df.loc[x.index, 'lead_id'][x].nunique())
    )
    .reset_index()
)

agg_df

,last_conversion_date,utm_campaign,utm_medium,utm_content,leads_unicos,lurq,luqmf
0,2025-12-20,120233998513890140,120233998513990140,120234078652600140,1,0,0
1,2025-12-20,120234014542220569,120234745949400569,120234745949430569,1,0,0
2,2025-12-20,120239868064490140,120239868064710140,120239868681980140,1,0,0
3,2025-12-20,120239868064490140,120239868064720140,120239868682640140,2,1,0
4,2025-12-20,120239868064490140,120239868064720140,120239868682650140,1,1,1
...,...,...,...,...,...,...,...
2351,2026-01-07,120240689426710140,120240689427220140,120240689426930140,3,1,0
2352,2026-01-07,120240689426710140,120240689427310140,120240689426800140,1,1,0
2353,2026-01-07,120240689426710140,120240689427310140,120240689427000140,4,3,1
2354,2026-01-07,lcto-ofan-jan26,social,link_in_bio,5,0,0


# Step 4: Merge com lookup table de campanhas, públicos e anúncios


## Step 4.1: Corrigindo tipos de Campanhas para melhor eficiência do Merge

In [7]:
# Convertendo para string para evitar erros de tipo
trf_data['date'] = trf_data['date'].astype(str)
trf_data['campaign_id'] = trf_data['campaign_id'].astype(str)
trf_data['adset_id'] = trf_data['adset_id'].astype(str)
trf_data['ad_id'] = trf_data['ad_id'].astype(str)

agg_df['last_conversion_date'] = agg_df['last_conversion_date'].astype(str)
agg_df['utm_campaign'] = agg_df['utm_campaign'].astype(str)
agg_df['utm_medium'] = agg_df['utm_medium'].astype(str)
agg_df['utm_content'] = agg_df['utm_content'].astype(str)


## Step 4.3: Importando Realizando Merges por UTM

In [9]:
A = trf_data.copy()
B = agg_df.copy()

for c in ["campaign_id", "adset_id", "ad_id"]:
    A[c] = A[c].astype("string")

for c in ["utm_campaign", "utm_medium", "utm_content"]:
    B[c] = B[c].astype("string")

left_keys  = ["date", "campaign_id", "adset_id", "ad_id"]
right_keys = ["last_conversion_date", "utm_campaign", "utm_medium", "utm_content"]

A_dup = A[A.duplicated(left_keys, keep=False)].sort_values(left_keys)
B_dup = B[B.duplicated(right_keys, keep=False)].sort_values(right_keys)

print("Duplicadas em A:", len(A_dup))
print("Duplicadas em B:", len(B_dup))

df_merged_utm = A.merge(
    B,
    left_on=["date", "campaign_id", "adset_id", "ad_id"],
    right_on=["last_conversion_date", "utm_campaign", "utm_medium", "utm_content"],
    how="left",
    validate="1:1",   # <- aqui garante 1:1
)

Duplicadas em A: 0
Duplicadas em B: 0


# Step 5: Limpeza e Finalização do Dataframe Final

In [9]:
def clean_data(df_merged_utm):
    # Drop duplicate rows in columns: 'utm_campaign', 'utm_medium', 'utm_content'
    df_merged_utm = df_merged_utm.drop_duplicates(subset=['utm_campaign', 'utm_medium', 'utm_content'])
    # Drop columns: 'campaign_id', 'adset_id', 'ad_id'
    df_merged_utm = df_merged_utm.drop(columns=['campaign_id', 'adset_id', 'ad_id'])
    # Drop rows with missing data in columns: 'campaign_name', 'adset_name', 'ad_name'
    df_merged_utm = df_merged_utm.dropna(subset=['campaign_name', 'adset_name', 'ad_name'], how='all')
    s = df_merged_utm["campaign_name"].astype("string")
    conditions = [
        s.str.contains("trabalho", case=False, na=False),
        s.str.contains("viagens", case=False, na=False),
        s.str.contains("cdf_vtsd_isca_aulas-semanais-yt", case=False, na=False),
    ]
    choices = ["trabalho", "viagens", "vtsd"]
    df_merged_utm["campaign_mote"] = np.select(conditions, choices, default="amplas")
    return df_merged_utm

df_final = clean_data(df_merged_utm.copy())
df_final.head()


,utm_campaign,utm_medium,utm_content,leads_unicos,lurq,luqmf,campaign_name,adset_name,ad_name,campaign_mote
0,120233326809280569,120233326809370569,120233326809290569,64,13,3,[aulas-semanais-yt] [LEADS] [CAPTAÇÃO] [JUL25]...,00 - [AUTO] LAL 1% Purchase 30D + Idade 25 a 5...,ID_00012_[MW][CDF][AULAS_SMN]_captacao_video_orig,amplas
388,120233326809280569,120233326809370569,120233328918040569,1,0,0,[aulas-semanais-yt] [LEADS] [CAPTAÇÃO] [JUL25]...,00 - [AUTO] LAL 1% Purchase 30D + Idade 25 a 5...,ID_00013_[MW][CDF][AULAS_SMN]_captacao_video_orig,amplas
776,120233326809280569,120233329509840569,120233329509800569,4,2,1,[aulas-semanais-yt] [LEADS] [CAPTAÇÃO] [JUL25]...,00 - [AUTO] [iOS] LAL 1% Purchase 30D + Idade ...,ID_00012_[MW][CDF][AULAS_SMN]_captacao_video_orig,amplas
1164,120233326809280569,120233329509840569,120233329509810569,22,6,2,[aulas-semanais-yt] [LEADS] [CAPTAÇÃO] [JUL25]...,00 - [AUTO] [iOS] LAL 1% Purchase 30D + Idade ...,ID_00015_[MW][CDF][AULAS_SMN]_captacao_video_orig,amplas
1552,120233326809280569,120233329509840569,120233329509830569,1,1,1,[aulas-semanais-yt] [LEADS] [CAPTAÇÃO] [JUL25]...,00 - [AUTO] [iOS] LAL 1% Purchase 30D + Idade ...,ID_00013_[MW][CDF][AULAS_SMN]_captacao_video_orig,amplas
